In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import mysql.connector
import logging

logging.basicConfig(filename="mysql.log", level=logging.INFO, format="%(asctime)s-%(levelname)s-%(message)s")

class MySQLDatabase:
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            self.cursor = self.connection.cursor()
            logging.info("Connection established")
        except Exception as e:
            logging.error(f"Failed to connect to MySQL database: {e}")
            print(f"Failed to connect to MySQL database: {e}")
    
    # you have to design different table for different method      
    def create_table(self, table_name, columns):
        self.table_name = table_name
        self.columns = columns
        try:
            self.cursor.execute(f"CREATE TABLE if not exists {self.table_name} ({self.columns})")
            self.connection.commit()
            logging.info("Table has been created")
            print("Table has been created")
        except Exception as e:
            logging.error("Exception occurred", e)
            
    def channel_details(self, table_name, channel_id):
        self.table_name = table_name
        driver = webdriver.Chrome()
        driver.get(f'https://www.youtube.com/@{channel_id}')
        channel_name = driver.find_element(By.CSS_SELECTOR,'#text-container').text
        subscribers = driver.find_element(By.CSS_SELECTOR, '#subscriber-count').text
        category = driver.find_element(By.CSS_SELECTOR, '#title-text').text
        show_more_button = driver.find_element(By.XPATH,'//*[@id="play-button"]/ytd-button-renderer/yt-button-shape/a')
        show_more_button.click()
        time.sleep(4)
        video_count = driver.find_elements(By.CSS_SELECTOR,'#publisher-container > div > yt-formatted-string > span:nth-child(3)')[0].text
        query = (f"INSERT INTO {self.table_name} VALUES (%s, %s, %s, %s)")
        try:
            values = (channel_name, subscribers, category, video_count)
            self.cursor.execute(query, values)
            rowscount = self.cursor.rowcount
            self.connection.commit()
            logging.info(f"Inserted {rowscount} document sucessfully")
            self.cursor.close()
            logging.info("Disconnected from MYSQL.")
        except Exception as e:
            logging.error(f"Failed to insert row into table: {e}")
            print(f"Failed to insert row into table: {e}")        
        driver.quit()
           
    def video_details(self, table_name, video_id):
        self.table_name = table_name
        driver = webdriver.Chrome()
        driver.get(f"https://www.youtube.com/watch?v={video_id}")
        show_more_button = driver.find_element(By.CSS_SELECTOR,'#expand')
        show_more_button.click()
        time.sleep(30)
        Likes = driver.find_element(By.CSS_SELECTOR,'#segmented-like-button > ytd-toggle-button-renderer > yt-button-shape > button > div.cbox.yt-spec-button-shape-next--button-text-content > span').text
        Views = driver.find_element(By.CSS_SELECTOR,'#info > span:nth-child(1)').text
        Title = driver.find_element(By.CSS_SELECTOR,'#title > h1 > yt-formatted-string').text
        Description = driver.find_elements(By.CSS_SELECTOR,'#description-inline-expander > yt-attributed-string > span')[0].text
        
        query = (f"INSERT INTO {self.table_name} VALUES (%s, %s, %s, %s)")
        try:
            values = (Title, Description, Likes, Views)
            self.cursor.execute(query, values)
            rowscount = self.cursor.rowcount
            self.connection.commit()
            logging.info(f"Inserted {rowscount} document sucessfully")
            self.cursor.close()
            logging.info("Disconnected from MYSQL.")
        except Exception as e:
            logging.error(f"Failed to insert row into table: {e}")
            print(f"Failed to insert row into table: {e}")

    def video_comments(self, video_id):
        try:
            driver = webdriver.Chrome()
            driver.get(f"https://www.youtube.com/watch?v={video_id}")
            #Get the height of the entire document
            last_height = driver.execute_script("return document.documentElement.scrollHeight")
            while True:
                # Scroll to the bottom of the page using JavaScript
                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
                time.sleep(4)
                new_height = driver.execute_script("return document.documentElement.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            usernames = driver.find_elements(By.CSS_SELECTOR, '#author-text')
            comments = driver.find_elements(By.CSS_SELECTOR, '#content-text')

            for i in range(len(usernames)):
                username = usernames[i].text
                comment = comments[i].text
                query = (f"INSERT INTO {self.table_name} VALUES (%s, %s)")
                values = (username, comment)
                self.cursor.execute(query, values)
                
            self.connection.commit()
            # Close the cursor
            self.cursor.close()
            logging.info("Comments upload successful")
            print("Comments upload successful")
        except Exception as e:
            logging.error(f"Failed to upload comments: {e}")
            print(f"Failed to upload comments: {e}")
        finally:
            # Close the browser
            driver.quit()


In [2]:
db = MySQLDatabase(hostname, your username, your password, database)

In [3]:
db.create_table("ytscrap", "username VARCHAR(100), comment VARCHAR(2000)")

Table has been created


In [4]:
db.video_comments(enter video id)

Comments upload successful


In [3]:
db.create_table("youtube", "TITLE VARCHAR(2000), DESCRIPTION VARCHAR(3000), LIKES VARCHAR(100), VIEWS VARCHAR(100)")

Table has been created


In [3]:
db.video_details("youtube", enter video id)

In [50]:
db.create_table("channel", "ChannelName VARCHAR(800), Subscribers VARCHAR(200), Category VARCHAR(500), TotalVideos VARCHAR(500)")

Table has been created


In [60]:
db.channel_details("channel", enter channel id)